In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
website = os.path.basename(os.path.dirname(os.path.realpath('__file__')))

In [ ]:
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.realpath('__file__')),'..'))

In [ ]:
from cache import Cache
import urllib, time
from civitatis.scraper import ActivityScraper, ReviewScraper
import pandas as pd
from helpers import create_download_link
from config import CSV_SEPARATOR, DT_FORMAT, URL_SHORTENER
from pathlib import Path

In [ ]:
home_page = 'https://www.civitatis.com'
path = 'es/puerto-de-la-cruz/'
url = urllib.parse.urljoin(home_page, path)
cache = Cache(expire_time=604800)
headers = {'User-Agent': 'XY'}
parser = 'lxml'

In [ ]:
s = ActivityScraper(url, cache, headers, parser)
s.run(f'Scraping {URL_SHORTENER.short(url)} ')

In [ ]:
df = s.get_data()
df.drop(['html'], axis=1).head(3)

In [ ]:
dfs = []
paths = df.url.to_list()
for activity_path in paths:
    url = urllib.parse.urljoin(home_page, activity_path) + 'opiniones'
    s = ReviewScraper(url, cache, headers, parser)
    s.run(f'Scraping {URL_SHORTENER.short(url)} ')
    df = s.get_data()
    dfs.append(df)

In [ ]:
pd.set_option('display.max_colwidth', None)
df = pd.concat(dfs)
df.drop('html', axis=1).head(3)

In [ ]:
output_file = f'{website}_{time.strftime(DT_FORMAT)}.csv'
create_download_link(df, filename=output_file, separator=CSV_SEPARATOR)

In [ ]:
output_file = str(Path.home() / "Downloads" / output_file)
pd.read_csv(output_file, engine='python', sep=CSV_SEPARATOR).drop('html', axis=1).sample(5).style.hide()